# Data Preprocessing - Test Space

In [176]:
# Enable Intellisense
%config IPCompleter.greedy=True

import pandas as pd
import numpy as np
import io
import requests
import seaborn as sns
from matplotlib import pyplot, dates
import plotly.express as px 
import datetime
import plotly.graph_objects as go
import plotly.io as pio
sns.set()

# Annoying warning
pd.options.mode.chained_assignment = None  # default='warn'

In [177]:
# ['EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW', 'WC']

## Gen Helper Functions

In [178]:
def df_from_url(df_url, pd_kwargs={}):
    df_req = requests.get(df_url).content
    df = pd.read_csv(io.StringIO(df_req.decode('utf-8')), **pd_kwargs)
    return df

In [179]:
tmp_kwargs = {"usecols":['date','total']}
df_from_url("https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_confirmed.csv", 
            tmp_kwargs).head()

,date,total
0,05-03-2020,1
1,07-03-2020,2
2,08-03-2020,3
3,09-03-2020,7
4,11-03-2020,13


## Fix for incomplete deaths data
'covid19za_provincial_cumulative_timeline_deaths.csv' includes all but the first 18 cases thus I will only be using the first 18 entries in `start_deaths_data`

## Start Data

In [180]:
start_deaths_data_url = "https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_timeline_deaths.csv"
start_deaths_data = df_from_url(start_deaths_data_url, pd_kwargs = {"usecols":['date','province']})
start_deaths_data = start_deaths_data.iloc[0:18] 
start_deaths_data['date'] = pd.to_datetime(start_deaths_data['date'], format='%d-%m-%Y')
start_deaths_data

,date,province
0,2020-03-27,WC
1,2020-03-28,KZN
2,2020-03-30,FS
3,2020-03-31,GP
4,2020-03-31,KZN
5,2020-04-03,KZN
6,2020-04-03,KZN
7,2020-04-03,KZN
8,2020-04-03,KZN
9,2020-04-05,WC


**Group by date and province and count occurrences**

In [181]:
start_deaths_by_prov = start_deaths_data.copy()
start_deaths_by_prov['tmp'] = 1
start_deaths_by_prov = start_deaths_by_prov.groupby(['date','province']).count()
start_deaths_by_prov.reset_index(inplace=True)
start_deaths_by_prov

,date,province,tmp
0,2020-03-27,WC,1
1,2020-03-28,KZN,1
2,2020-03-30,FS,1
3,2020-03-31,GP,1
4,2020-03-31,KZN,1
5,2020-04-03,KZN,4
6,2020-04-05,KZN,1
7,2020-04-05,WC,1
8,2020-04-06,WC,1
9,2020-04-07,KZN,1


In [182]:
tmp_pivot = start_deaths_by_prov.pivot(index='date', columns='province', values=['tmp'])
tmp_pivot

tmp               
province     FS   GP  KZN   WC
date                          
2020-03-27  NaN  NaN  NaN  1.0
2020-03-28  NaN  NaN  1.0  NaN
2020-03-30  1.0  NaN  NaN  NaN
2020-03-31  NaN  1.0  1.0  NaN
2020-04-03  NaN  NaN  4.0  NaN
2020-04-05  NaN  NaN  1.0  1.0
2020-04-06  NaN  NaN  NaN  1.0
2020-04-07  NaN  NaN  1.0  NaN
2020-04-08  2.0  2.0  1.0  NaN

In [183]:
tmp_pivot.columns = tmp_pivot.columns.droplevel(level=0)
tmp_pivot.columns.name = ""
tmp_pivot.reset_index(inplace = True)
tmp_pivot

,date,FS,GP,KZN,WC
0,2020-03-27,NaN,NaN,NaN,1.0
1,2020-03-28,NaN,NaN,1.0,NaN
2,2020-03-30,1.0,NaN,NaN,NaN
3,2020-03-31,NaN,1.0,1.0,NaN
4,2020-04-03,NaN,NaN,4.0,NaN
5,2020-04-05,NaN,NaN,1.0,1.0
6,2020-04-06,NaN,NaN,NaN,1.0
7,2020-04-07,NaN,NaN,1.0,NaN
8,2020-04-08,2.0,2.0,1.0,NaN


In [184]:
tmp_pivot.fillna(0, inplace=True)
tmp_pivot

,date,FS,GP,KZN,WC
0,2020-03-27,0.0,0.0,0.0,1.0
1,2020-03-28,0.0,0.0,1.0,0.0
2,2020-03-30,1.0,0.0,0.0,0.0
3,2020-03-31,0.0,1.0,1.0,0.0
4,2020-04-03,0.0,0.0,4.0,0.0
5,2020-04-05,0.0,0.0,1.0,1.0
6,2020-04-06,0.0,0.0,0.0,1.0
7,2020-04-07,0.0,0.0,1.0,0.0
8,2020-04-08,2.0,2.0,1.0,0.0


Data required is cumulative deaths not daily deaths thus need to apply cumsum()

In [185]:
tmp_pivot[['FS','GP','KZN','WC']] = tmp_pivot[['FS','GP','KZN','WC']].cumsum()
tmp_pivot.head()

,date,FS,GP,KZN,WC
0,2020-03-27,0.0,0.0,0.0,1.0
1,2020-03-28,0.0,0.0,1.0,1.0
2,2020-03-30,1.0,0.0,1.0,1.0
3,2020-03-31,1.0,1.0,2.0,1.0
4,2020-04-03,1.0,1.0,6.0,1.0


In [186]:
def change_date_format(in_date):
    return in_date.strftime('%Y%m%d')
change_date_format(datetime.datetime.strptime('08-04-2020', '%d-%m-%Y'))

'20200408'

In [187]:
tmp_pivot['YYYYMMDD'] = tmp_pivot['date'].apply(change_date_format)
tmp_pivot

,date,FS,GP,KZN,WC,YYYYMMDD
0,2020-03-27,0.0,0.0,0.0,1.0,20200327
1,2020-03-28,0.0,0.0,1.0,1.0,20200328
2,2020-03-30,1.0,0.0,1.0,1.0,20200330
3,2020-03-31,1.0,1.0,2.0,1.0,20200331
4,2020-04-03,1.0,1.0,6.0,1.0,20200403
5,2020-04-05,1.0,1.0,7.0,2.0,20200405
6,2020-04-06,1.0,1.0,7.0,3.0,20200406
7,2020-04-07,1.0,1.0,8.0,3.0,20200407
8,2020-04-08,3.0,3.0,9.0,3.0,20200408


In [188]:
tmp_pivot['total'] = tmp_pivot[['FS','GP','KZN','WC']].sum(axis=1)
start_data = tmp_pivot.copy()
start_data[['FS','GP','KZN','WC','total']] = start_data[['FS','GP','KZN','WC','total']].astype('int32')
start_data

,date,FS,GP,KZN,WC,YYYYMMDD,total
0,2020-03-27,0,0,0,1,20200327,1
1,2020-03-28,0,0,1,1,20200328,2
2,2020-03-30,1,0,1,1,20200330,3
3,2020-03-31,1,1,2,1,20200331,5
4,2020-04-03,1,1,6,1,20200403,9
5,2020-04-05,1,1,7,2,20200405,11
6,2020-04-06,1,1,7,3,20200406,12
7,2020-04-07,1,1,8,3,20200407,13
8,2020-04-08,3,3,9,3,20200408,18


## End Data
Note: First row of this data is ommited as it is included in `start_data`

In [189]:
end_data_url = "https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_deaths.csv"
end_data = df_from_url(end_data_url).iloc[1:]
end_data['date'] = pd.to_datetime(end_data['date'], format='%d-%m-%Y')
end_data

,date,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total
1,2020-04-09,20200409,0,3,3,9,0,0,0,0,3,0,18
2,2020-04-10,20200410,0,3,3,12,0,0,0,0,6,0,24
3,2020-04-11,20200411,0,3,3,12,0,0,0,0,7,0,25
4,2020-04-12,20200412,0,3,3,12,0,0,0,0,7,0,25
5,2020-04-13,20200413,0,3,4,12,0,0,0,0,8,0,27
6,2020-04-14,20200414,0,3,4,12,0,0,0,0,8,0,27
7,2020-04-15,20200415,0,3,5,18,0,0,0,0,8,0,34
8,2020-04-16,20200416,4,4,6,20,1,0,0,0,13,0,48
9,2020-04-17,20200417,4,4,6,20,1,0,0,0,15,0,50
10,2020-04-18,20200418,4,4,6,21,1,0,0,0,16,0,52


### Concat  start_data and end_data

In [190]:
all_data = pd.concat([start_data,end_data]).fillna(0)
col_ordered = ['date','YYYYMMDD','EC','FS','GP','KZN','LP','MP','NC','NW','WC','UNKNOWN','total']
# reorder columns
all_data = all_data[col_ordered]
all_data.head()

,date,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total
0,2020-03-27,20200327,0.0,0,0,0,0.0,0.0,0.0,0.0,1,0.0,1
1,2020-03-28,20200328,0.0,0,0,1,0.0,0.0,0.0,0.0,1,0.0,2
2,2020-03-30,20200330,0.0,1,0,1,0.0,0.0,0.0,0.0,1,0.0,3
3,2020-03-31,20200331,0.0,1,1,2,0.0,0.0,0.0,0.0,1,0.0,5
4,2020-04-03,20200403,0.0,1,1,6,0.0,0.0,0.0,0.0,1,0.0,9


In [191]:
# Convert all nos to int
all_data.iloc[:,2:] = all_data.iloc[:,2:].astype('int32')
all_data

,date,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total
0,2020-03-27,20200327,0,0,0,0,0,0,0,0,1,0,1
1,2020-03-28,20200328,0,0,0,1,0,0,0,0,1,0,2
2,2020-03-30,20200330,0,1,0,1,0,0,0,0,1,0,3
3,2020-03-31,20200331,0,1,1,2,0,0,0,0,1,0,5
4,2020-04-03,20200403,0,1,1,6,0,0,0,0,1,0,9
5,2020-04-05,20200405,0,1,1,7,0,0,0,0,2,0,11
6,2020-04-06,20200406,0,1,1,7,0,0,0,0,3,0,12
7,2020-04-07,20200407,0,1,1,8,0,0,0,0,3,0,13
8,2020-04-08,20200408,0,3,3,9,0,0,0,0,3,0,18
1,2020-04-09,20200409,0,3,3,9,0,0,0,0,3,0,18


### Add my latest deaths per prov data

In [192]:
latest_day_data = pd.read_csv('tot_deaths_provinces.csv')
latest_day_data

,province,tot_deaths
0,Eastern Cape,11
1,Free State,5
2,Gauteng,11
3,KwaZulu-Natal,32
4,Limpopo,2
5,Mpumalanga,0
6,Northern Cape,0
7,North West,0
8,Western Cape,42
9,Unknown,0


In [193]:
prov_map = {
    "Eastern Cape":"EC",
    "Free State":"FS",
    "Gauteng":"GP",
    "KwaZulu-Natal":"KZN",
    "Limpopo":"LP",
    "Mpumalanga":"MP",
    "Northern Cape":"NC",
    "North West":"NW",
    "Western Cape":"WC",
    "Unknown":"UNKNOWN"
}

In [194]:
latest_day_data['province'] = latest_day_data['province'].map(prov_map)
latest_day_data

,province,tot_deaths
0,EC,11
1,FS,5
2,GP,11
3,KZN,32
4,LP,2
5,MP,0
6,NC,0
7,NW,0
8,WC,42
9,UNKNOWN,0


In [195]:
latest_day_data['date'] = "29-04-2020"
latest_day_data['date'] = pd.to_datetime(latest_day_data['date'], format='%d-%m-%Y')
latest_day_data

,province,tot_deaths,date
0,EC,11,2020-04-29
1,FS,5,2020-04-29
2,GP,11,2020-04-29
3,KZN,32,2020-04-29
4,LP,2,2020-04-29
5,MP,0,2020-04-29
6,NC,0,2020-04-29
7,NW,0,2020-04-29
8,WC,42,2020-04-29
9,UNKNOWN,0,2020-04-29


In [196]:
latest_day_data_piv = latest_day_data.pivot(index='date', columns='province', values=['tot_deaths'])
latest_day_data_piv

tot_deaths                                   
province           EC FS  GP KZN LP MP NC NW UNKNOWN  WC
date                                                    
2020-04-29         11  5  11  32  2  0  0  0       0  42

In [197]:
latest_day_data_piv.columns = latest_day_data_piv.columns.droplevel(level=0)
latest_day_data_piv.columns.name = ""
latest_day_data_piv.reset_index(inplace = True)
latest_day_data_piv

,date,EC,FS,GP,KZN,LP,MP,NC,NW,UNKNOWN,WC
0,2020-04-29,11,5,11,32,2,0,0,0,0,42


In [198]:
latest_day_data_piv['total'] = latest_day_data_piv.drop(['date'], axis=1).sum(axis=1)
latest_day_data_piv

,date,EC,FS,GP,KZN,LP,MP,NC,NW,UNKNOWN,WC,total
0,2020-04-29,11,5,11,32,2,0,0,0,0,42,103


In [199]:
latest_day_data_piv['YYYYMMDD'] = latest_day_data_piv['date'].apply(change_date_format)
latest_day_data_piv

,date,EC,FS,GP,KZN,LP,MP,NC,NW,UNKNOWN,WC,total,YYYYMMDD
0,2020-04-29,11,5,11,32,2,0,0,0,0,42,103,20200429


In [200]:
all_data_latest = pd.concat([all_data, latest_day_data_piv])
all_data_latest

,date,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total
0,2020-03-27,20200327,0,0,0,0,0,0,0,0,1,0,1
1,2020-03-28,20200328,0,0,0,1,0,0,0,0,1,0,2
2,2020-03-30,20200330,0,1,0,1,0,0,0,0,1,0,3
3,2020-03-31,20200331,0,1,1,2,0,0,0,0,1,0,5
4,2020-04-03,20200403,0,1,1,6,0,0,0,0,1,0,9
5,2020-04-05,20200405,0,1,1,7,0,0,0,0,2,0,11
6,2020-04-06,20200406,0,1,1,7,0,0,0,0,3,0,12
7,2020-04-07,20200407,0,1,1,8,0,0,0,0,3,0,13
8,2020-04-08,20200408,0,3,3,9,0,0,0,0,3,0,18
1,2020-04-09,20200409,0,3,3,9,0,0,0,0,3,0,18


### Duplicate data for 29 April and set date to 30 April
No deaths were recorded for 30 April thus data will the same as for 29 April

In [201]:
curr_day = all_data_latest.tail(1)
curr_day['date'] = "30-04-2020"
curr_day['date'] = pd.to_datetime(curr_day['date'], format='%d-%m-%Y')
all_data_latest = pd.concat([all_data_latest, curr_day])
all_data_latest

,date,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total
0,2020-03-27,20200327,0,0,0,0,0,0,0,0,1,0,1
1,2020-03-28,20200328,0,0,0,1,0,0,0,0,1,0,2
2,2020-03-30,20200330,0,1,0,1,0,0,0,0,1,0,3
3,2020-03-31,20200331,0,1,1,2,0,0,0,0,1,0,5
4,2020-04-03,20200403,0,1,1,6,0,0,0,0,1,0,9
5,2020-04-05,20200405,0,1,1,7,0,0,0,0,2,0,11
6,2020-04-06,20200406,0,1,1,7,0,0,0,0,3,0,12
7,2020-04-07,20200407,0,1,1,8,0,0,0,0,3,0,13
8,2020-04-08,20200408,0,3,3,9,0,0,0,0,3,0,18
1,2020-04-09,20200409,0,3,3,9,0,0,0,0,3,0,18


### Fix format of date so its consistent with original data

In [202]:
all_data_latest['date'] = all_data_latest['date'].apply(lambda x: x.strftime('%d-%m-%Y'))
all_data_latest.tail()

,date,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total
18,26-04-2020,20200426,10,5,8,29,2,0,0,0,33,0,87
19,27-04-2020,20200427,10,5,8,30,2,0,0,0,35,0,90
20,28-04-2020,20200428,10,5,8,30,2,0,0,0,38,0,93
0,29-04-2020,20200429,11,5,11,32,2,0,0,0,42,0,103
0,30-04-2020,20200429,11,5,11,32,2,0,0,0,42,0,103


In [203]:
all_data_latest.to_csv('covid19za_provincial_cumulative_timeline_deaths.csv', index=False)

In [204]:
pd.read_csv('covid19za_provincial_cumulative_timeline_deaths.csv')

,date,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total
0,27-03-2020,20200327,0,0,0,0,0,0,0,0,1,0,1
1,28-03-2020,20200328,0,0,0,1,0,0,0,0,1,0,2
2,30-03-2020,20200330,0,1,0,1,0,0,0,0,1,0,3
3,31-03-2020,20200331,0,1,1,2,0,0,0,0,1,0,5
4,03-04-2020,20200403,0,1,1,6,0,0,0,0,1,0,9
5,05-04-2020,20200405,0,1,1,7,0,0,0,0,2,0,11
6,06-04-2020,20200406,0,1,1,7,0,0,0,0,3,0,12
7,07-04-2020,20200407,0,1,1,8,0,0,0,0,3,0,13
8,08-04-2020,20200408,0,3,3,9,0,0,0,0,3,0,18
9,09-04-2020,20200409,0,3,3,9,0,0,0,0,3,0,18
